# Computing the convex hull of 2 points in d-space

Following the book 'Introduction to Tropical Goemetry', Prop. 5.2.5

First, we copy some stuff from the other sage code that allows us to compute everything symbolically

In [78]:
a,b,c,d,e,f = var('a,b,c,d,e,f')

In [79]:
def symbols_to_vector(symbolic_expression):
    symbols_vec = [0,a,b,c,d,e,f]
    string = str(symbolic_expression)
    vec = [0,0,0,0,0,0,0]
    sign = 1
    if string == "0":
        return vector(vec)
    for letter in string:
        if letter in [" "]:
            continue
        elif letter in ["+"]:
            sign = 1
        elif letter in ["-"]:
            sign = -1
        elif var(letter) in symbols_vec:
            index = symbols_vec.index(var(letter))
            vec[index] = 1*sign
    return vector(vec)

class expression:
    
    def __init__(self, coordinates, assumptions):
        self.assumptions = assumptions
        self.coordinates = vector(coordinates)
        
    def is_valid_strict(self):
        #checks if a strict inequality (question) is satified under the assumptions of other strict inequalities
        #this is not satisfied if intP interected with H_c is nonempty
        P = Polyhedron(ieqs = self.assumptions)
        #complement of halfspace defined by strict inequality in question
        H_c = Polyhedron(ieqs = [[-i for i in self.coordinates]]) 
        if len(H_c.intersection(P).vertices())==0:
            return true
        else:
            for F in P.faces(5):
                if H_c.intersection(F.as_polyhedron()) == H_c.intersection(P):
                    return true
        return false
    
    def is_valid_nonstrict(self):
        #checks if a non-strict inequality (question) is satified under the assumptions of other strict inequalities
        #this is satisfied if intP is contained in H. H is closed, so this is equivalent to P contianed in H
        P = Polyhedron(ieqs = self.assumptions)
        H = Polyhedron(ieqs = [self.coordinates]) 
        if H.intersection(P)==P:
            return true
        else:     
            return false

    def  __gt__(self, other):
        #definition of >
        #return self.is_valid_strict(assumptions, self.coordinates - other.coordinates)
        return expression(coordinates = self.coordinates - other.coordinates, assumptions = self.assumptions).is_valid_strict()
    
    def __lt__(self,other):
        #definition of <
        return expression(coordinates = other.coordinates - self.coordinates, assumptions = self.assumptions).is_valid_strict()
    
    def __eq__(self, other):
        #definition of ==
        if expression(coordinates = self.coordinates - other.coordinates, assumptions = self.assumptions).is_valid_nonstrict() and expression(coordinates = other.coordinates - self.coordinates, assumptions = self.assumptions).is_valid_nonstrict(): 
            return true
        else:
            return false
    
    def __le__(self, other):
        #definition of <=
        return (self<other or self==other)
    
    def __ge__(self, other):
        #definition of >=
        return (self>other or self==other)
    
    def __add__(self,other):
        #definition of +
        return expression(coordinates = self.coordinates + other.coordinates, assumptions = self.assumptions)
    
    def __sub__(self,other):
        #definition of -
        return expression(coordinates = self.coordinates - other.coordinates, assumptions = self.assumptions)
    
    def is_comparable_to(self, other):
        if self>= other or self <= other:
            return true
        else:
            return false
        
    def as_symbols(self):
        symbols_vec = vector([0,a,b,c,d,e,f])
        return symbols_vec.dot_product(vector(self.coordinates))

As test set, we take the first feasible region in the Kleene fan, which corresponds to the Kleene star
$\begin{pmatrix}
0 & a & b \\
c & 0 & d \\
e & f & 0
\end{pmatrix}$.

Thus, the corresponding polytrope has tropical vertices $(a,b), (-c, d-c), (f-e, -e)$.

In [80]:
region_ieqs = [[0, -1, 1, 0, 0, 0, 1],
 [0, 0, 0, -1, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, -1, 1],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 1, -1, 0, 0],
 [0, 1, -1, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, -1]]
vertices = [[a,b], [-c, d-c], [f-e, -e]]

In [81]:
vertex1 = vertices[0]
vertex2 = vertices[1]
print vertex1, vertex2

[a, b] [-c, -c + d]


For normalization purposes, we have to add a 0 as first coordinate

In [82]:
v1 = vector([0]+list(vertex1))
v2 = vector([0]+list(vertex2))
length = len(list(v1))
print v1, v2

(0, a, b) (0, -c, -c + d)


We want to find a permutation $\pi$ of indices such that $v2_{\pi(i)}-v1_{\pi(i)}\leq v2_{\pi(i+1)}-v1_{\pi(i+1)}$ for all indices $i$

In [83]:
pairs = [[v2[i]-v1[i], i] for i in range(length)]
pairs

[[0, 0], [-a - c, 1], [-b - c + d, 2]]

In [84]:
sorted_ex = sorted([[expression(assumptions = region_ieqs, coordinates = symbols_to_vector(el[0])), el[1]] for el in pairs],key=lambda x: x[0])
[[el[0].as_symbols(), el[1]] for el in sorted_ex]

[[-a - c, 1], [-b - c + d, 2], [0, 0]]

Following Prop. 5.2.5, we can explicitly write down the points on the line

In [85]:
pseudovertices = []
for i in range(1,length-1):
    current = permutation[i]
    pseudo = [0]*length
    for index in permutation[:i]:
        pseudo[index] = v2[index]
    for index in permutation[i:]:
        pseudo[index] = v2[current] - v1[current] + v1[index]
    pseudovertices.append(pseudo)
print pseudovertices

[[-a - c, -c, -c + d]]


Normalize by setting the first coordinate to zero

In [86]:
for point in pseudovertices:
    normalization = vector(pseudo) - vector([pseudo[0]]*length)
    print normalization

(0, a, a + d)


In [89]:
def pseudovertices_on_line_symb(vertex1,vertex2,assumpt):
    #add zero for normalization purposes
    v1 = vector([0]+list(vertex1))
    v2 = vector([0]+list(vertex2))
    length = len(list(v1))
    #find permutation p of indices s.t. v2[p(i)]-v1[p(i)]<=v2[p(i+1)]-v1[p(i+1)]
    pairs = [[v2[i]-v1[i], i] for i in range(length)]
    sorted_ex = sorted([[expression(assumptions = assumpt, coordinates = symbols_to_vector(el[0])), el[1]] for el in pairs],key=lambda x: x[0])
    permutation = [x[1] for x in sorted_ex]
    #compute inner pseudovertices on line segment
    pseudovertices = []
    for i in range(1,length-1):
        current = permutation[i]
        pseudo = [0]*length
        for index in permutation[:i]:
            pseudo[index] = v2[index]
        for index in permutation[i:]:
            pseudo[index] = v2[current] - v1[current] + v1[index]
        normalization = vector(pseudo) - vector([pseudo[0]]*length)
        pseudovertices.append(list(normalization[1:]))
    return pseudovertices

In [90]:
for pair in Subsets(range(len(vertices)),2):
    print pseudovertices_on_line_symb(vertices[pair[0]],vertices[pair[1]],region_ieqs)

[[b - d, b]]
[[a, a - f]]
[[-c, -e]]


The same works for actual numbers:

In [91]:
def pseudovertices_on_line(vertex1,vertex2):
    #add zero for normalization purposes
    v1 = vector([0]+list(vertex1))
    v2 = vector([0]+list(vertex2))
    length = len(list(v1))
    #find permutation p of indices s.t. v2[p(i)]-v1[p(i)]<=v2[p(i+1)]-v1[p(i+1)]
    pairs = [[v2[i]-v1[i], i] for i in range(length)]
    permutation = [x[1] for x in sorted(pairs,key=lambda x: x[0])]
    #compute inner pseudovertices on line segment
    pseudovertices = []
    for i in range(1,length-1):
        current = permutation[i]
        pseudo = [0]*length
        for index in permutation[:i]:
            pseudo[index] = v2[index]
        for index in permutation[i:]:
            pseudo[index] = v2[current] - v1[current] + v1[index]
        normalization = vector(pseudo) - vector([pseudo[0]]*length)
        pseudovertices.append(list(normalization[1:]))
    return pseudovertices

In [93]:
pseudovertices_on_line([0,0,0,0,0],[1,2,3,4,5])

[[1, 1, 1, 1, 1], [1, 2, 2, 2, 2], [1, 2, 3, 3, 3], [1, 2, 3, 4, 4]]